<a href="https://colab.research.google.com/github/LashawnFofung/RAG-Pipelines/blob/main/Gradio/Task_Gradio_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gradio Chatbot**

**GOAL:** A document Q&A interface with a sleek monochrome theme.

<br>

**PDF Research Buddy: Project Submission**

This notebook contains a Gradio-based chatbot designed to analyze PDF documents using a monochrome theme and a custom "Analysis Mode" feature.

<br>

**FEATURES:**
1. Mode Selection (Dropdown) to switch between Q&A and Summarization.
2. Visual Loading Status via a dedicated status box and progress bars.

# **1. Setup & Installation**

In [1]:
!pip install -q gradio

# **2. Main Application (Frontend & Backend Logic)**

In [3]:
#Frontend & Backend Logic in single codeblock,
# so when code block run the entire app launches at once to prevent "Variable Not Defined" errors.

import gradio as gr
import time

# --- 1. LOGIC FUNCTIONS ---

def process_pdf(file):
    if file is None:
        return "⚠️ Error: Please upload a PDF file."
    time.sleep(1.5)
    return f"COMPLETED\n---\nDocument: {file.name.split('/')[-1]}\nStatus: Ready."

def handle_chat(message, history, mode):
    if not message:
        return "", history

    responses = {
        "Summary": "The document provides an overview of technical implementation strategies...",
        "Key Takeaways": "1. Efficiency is prioritized.\n2. User design is critical.\n3. Scalability is the goal.",
        "Standard Q&A": f"Analyzing the text for '{message}'... The data indicates a strong positive trend."
    }

    bot_response = responses.get(mode, "Mode not recognized.")

    # Using the modern 'messages' format
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": bot_response})

    return "", history

# --- 2. THEME & STYLING ---

monochrome_theme = gr.themes.Soft(
    primary_hue="slate",
    secondary_hue="gray",
    neutral_hue="slate",
).set(
    button_primary_background_fill="*neutral_900",
    button_primary_text_color="white",
    block_title_text_weight="700",
)

# Defined here to be used in gr.Blocks
custom_css = "#status_box { background-color: #f8fafc; border: 1px solid #cbd5e1; }"

# --- 3. UI LAYOUT ---

# Use a clean approach here to avoid loop conflicts
gr.close_all()

#  Both theme and css here to avoid the TypeError in launch()
with gr.Blocks(theme=monochrome_theme, css=custom_css, title="AI Research Buddy") as demo:
    gr.Markdown("# 📑 PDF Research Buddy")

    with gr.Row():
        with gr.Column(scale=2, min_width=400):
            gr.Markdown("#### ⚙️ Input & Settings")
            pdf_input = gr.File(label="Upload Document", file_types=[".pdf"])

            mode_select = gr.Dropdown(
                choices=["Standard Q&A", "Summary", "Key Takeaways"],
                value="Standard Q&A",
                label="Analysis Mode"
            )

            process_btn = gr.Button("🔄 Process Document", variant="primary")

            status_output = gr.Textbox(
                label="System Status & Metadata",
                interactive=False,
                lines=6,
                elem_id="status_box",
                placeholder="System idle..."
            )

            exit_btn = gr.Button("🚪 Exit & Shut Down", variant="stop")

        with gr.Column(scale=3):
            gr.Markdown("#### 💬 Document Chat")
            # Added allow_tags=False to resolve the final warning
            chatbot = gr.Chatbot(
                label="Conversation History",
                height=500,
                type="messages",
                allow_tags=False
            )

            user_input = gr.Textbox(
                placeholder="Ask a question...",
                label="User Query"
            )

            with gr.Row():
                send_btn = gr.Button("📤 Send", variant="primary")
                clear_btn = gr.Button("🗑️ Clear Chat")

    # --- 4. EVENT LISTENERS ---

    process_btn.click(fn=process_pdf, inputs=pdf_input, outputs=status_output)

    send_btn.click(handle_chat, inputs=[user_input, chatbot, mode_select], outputs=[user_input, chatbot])
    user_input.submit(handle_chat, inputs=[user_input, chatbot, mode_select], outputs=[user_input, chatbot])

    clear_btn.click(lambda: [], None, chatbot, queue=False)

    exit_btn.click(lambda: gr.Info("Shutting down..."), None, None).then(fn=demo.close)

# --- 5. LAUNCH ---
if __name__ == "__main__":
    # Removed theme and css from here because your version doesn't support them in launch()
    demo.launch(debug=True)

Closing server running on port: 7860


/tmp/ipython-input-1434651072.py:54: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=monochrome_theme, css=custom_css, title="AI Research Buddy") as demo:
/tmp/ipython-input-1434651072.py:54: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=monochrome_theme, css=custom_css, title="AI Research Buddy") as demo:
/tmp/ipython-input-1434651072.py:83: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2b82f41959c9ba9f81.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Closing server running on port: 7860
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2b82f41959c9ba9f81.gradio.live
